In [1]:
import argparse
import json
import logging
import pickle
import random
import time
from copy import deepcopy
from dataclasses import dataclass, field, fields
from datetime import datetime, timedelta, timezone
from functools import partial
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import yaml
from pathos.multiprocessing import ProcessingPool as Pool
from sbi.inference import NPE
from sbi.utils import BoxUniform, RestrictedPrior, get_density_thresholder
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_simulator,
)
from simglucose.actuator.pump import InsulinPump
from simglucose.controller.basal_bolus_ctrller import BBController
from simglucose.patient.t1dpatient import T1DPatient
from simglucose.sensor.cgm import CGMSensor
from simglucose.simulation.env import T1DSimEnv
from simglucose.simulation.scenario import CustomScenario
from simglucose.simulation.sim_engine import SimObj
from sklearn.metrics import mean_squared_error
from torch.distributions import Distribution
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def sample_from_posterior(posterior: Distribution, x_true: np.ndarray, num_samples: int = 1000) -> torch.Tensor: 
    x_true = torch.tensor(x_true, dtype=torch.float32, device=device)
    return posterior.sample((num_samples,), x=x_true)

In [4]:
from infer_parameters import DeafultSimulationEnv, ParamPriors

In [16]:
def load_experimental_setup(results_folder: str) -> tuple[DeafultSimulationEnv, ParamPriors, np.ndarray, np.ndarray, Distribution]:
    """Loads the experimental setup and the results of the inference.

    Parameters
    ----------
    results_folder : str
        Path to the folder containing the results of the inference.

    Returns
    -------
    tuple[DeafultSimulationEnv, ParamPriors, np.ndarray, np.ndarray, Distribution]
        The experimental setup and the results of the inference.

    """
    setup_folder = Path(results_folder) / "Experimental Setup"
    with (setup_folder / "default_settings.pkl").open("rb") as f:
        default_settings = pickle.load(f)
    with (setup_folder / "priors.pkl").open("rb") as f:
        priors = pickle.load(f)
    with (setup_folder / "true_observation.pkl").open("rb") as f:
        true_observation = pickle.load(f)
    with (setup_folder / "true_params.pkl").open("rb") as f:
        true_params = pickle.load(f)
    with (Path(results_folder) / "posterior_distribution.pkl").open("rb") as f:
        posterior_distribution = pickle.load(f)
    return default_settings, priors, true_observation, true_params, posterior_distribution

In [17]:
default_settings, priors, true_observation, true_params, posterior_distribution = load_experimental_setup("results/2025-02-05_18-58")

In [ ]:
samples = sample_from_posterior(posterior_distribution, true_observation, num_samples=10)

In [19]:
import infer_parameters as ip

glucose_dynamics = ip.run_glucose_simulator(params_sets=samples, default_settings=default_settings, priors=priors, device=device)

In [20]:
glucose_dynamics = glucose_dynamics.cpu().numpy()
true_observation = true_observation.cpu().numpy()
mean_glucose_dynamics = np.mean(glucose_dynamics, axis=0)
std_glucose_dynamics = np.std(glucose_dynamics, axis=0)

In [ ]:
plt.plot(mean_glucose_dynamics, label="Mean glucose dynamics")
plt.fill_between(range(len(mean_glucose_dynamics)), mean_glucose_dynamics - std_glucose_dynamics, mean_glucose_dynamics + std_glucose_dynamics, alpha=0.5, label="Standard deviation")  
plt.plot(true_observation, label="True observation")

In [15]:

patient = T1DPatient.withName("adolescent#001")

In [16]:
print(type(patient._params))
patient._params.to_csv("params.csv")
param_dict = patient._params.to_dict()

<class 'pandas.core.series.Series'>


In [17]:
print(param_dict)

{'Name': 'adolescent#001', 'i': 1, 'x0_ 1': 0, 'x0_ 2': 0, 'x0_ 3': 0, 'x0_ 4': 250.621836, 'x0_ 5': 176.506559902, 'x0_ 6': 4.697517762, 'x0_ 7': 0, 'x0_ 8': 97.554, 'x0_ 9': 97.554, 'x0_10': 3.19814917262, 'x0_11': 57.951224472, 'x0_12': 93.2258828462, 'x0_13': 250.621836, 'BW': 68.706, 'EGPb': 3.3924, 'Gb': 149.02, 'Ib': 97.554, 'kabs': 0.091043, 'kmax': 0.015865, 'kmin': 0.0083338, 'b': 0.83072, 'd': 0.32294, 'Vg': 1.6818, 'Vi': 0.048153, 'Ipb': 4.697517762, 'Vmx': 0.074667, 'Km0': 260.89, 'k2': 0.067738, 'k1': 0.057252, 'p2u': 0.021344, 'm1': 0.15221, 'm5': 0.029902, 'CL': 0.8571, 'HEb': 0.6, 'm2': 0.259067825939, 'm4': 0.103627130376, 'm30': 0.228315, 'Ilb': 3.19814917262, 'ki': 0.0088838, 'kp2': 0.023318, 'kp3': 0.023253, 'f': 0.9, 'Gpb': 250.621836, 'ke1': 0.0005, 'ke2': 339, 'Fsnc': 1, 'Gtb': 176.506559902, 'Vm0': 5.92854753098, 'Rdb': 3.3924, 'PCRb': 0.0227647295665, 'kd': 0.0185, 'ksc': 0.056, 'ka1': 0.0025, 'ka2': 0.0115, 'dosekempt': 90000, 'u2ss': 1.21697571391, 'isc1ss':

In [18]:
for key, value in param_dict.items():
    param_dict[key] = [value]

In [19]:
def make_positive_definite(matrix, epsilon=1e-6):
    """Add a small value to the diagonal elements to make the matrix positive-definite."""
    return matrix + epsilon * np.eye(matrix.shape[0])

In [20]:
ixd = ["02", "03", "04", "05", "06", "07", "08", "09", "10"]
for i in ixd:
    print(f"adolescent#0{i}")
    patient = T1DPatient.withName(f"adolescent#0{i}")
    for key, value in patient._params.to_dict().items():
        param_dict[key].append(value)
   

adolescent#002
adolescent#003
adolescent#004
adolescent#005
adolescent#006
adolescent#007
adolescent#008
adolescent#009
adolescent#010


In [21]:
print(param_dict)

{'Name': ['adolescent#001', 'adolescent#002', 'adolescent#003', 'adolescent#004', 'adolescent#005', 'adolescent#006', 'adolescent#007', 'adolescent#008', 'adolescent#009', 'adolescent#010'], 'i': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'x0_ 1': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'x0_ 2': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'x0_ 3': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'x0_ 4': [250.621836, 280.236267, 326.55338, 248.11709, 279.047113, 257.421705, 238.918844, 263.83812, 241.4475, 267.183741], 'x0_ 5': [176.506559902, 30.9541309204, 383.368339011, 214.758607646, 144.715758769, 162.558570565, 87.9615903151, 33.9274151265, 184.846172962, 169.7110728], 'x0_ 6': [4.697517762, 6.01930508, 3.48788064, 4.31676536, 5.16289777, 8.154419, 5.2342344, 4.730160786, 4.707832836, 6.32186544], 'x0_ 7': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'x0_ 8': [97.554, 119.18, 101.28, 108.88, 113.09, 113.0, 111.9, 82.914, 84.982, 108.24], 'x0_ 9': [97.554, 119.18, 101.28, 108.88, 113.09, 113.0, 111.9, 82.914, 84.982, 108.24], 'x0_10': [3.1

In [22]:
# remove key "name"
for key in ["Name", "x0_ 1", "x0_ 2", "x0_ 3", "x0_ 7", "f", "ke1", "ke2", "Fsnc", "dosekempt", "patient_history", "i"]:
    param_dict.pop(key)
print(param_dict)


{'x0_ 4': [250.621836, 280.236267, 326.55338, 248.11709, 279.047113, 257.421705, 238.918844, 263.83812, 241.4475, 267.183741], 'x0_ 5': [176.506559902, 30.9541309204, 383.368339011, 214.758607646, 144.715758769, 162.558570565, 87.9615903151, 33.9274151265, 184.846172962, 169.7110728], 'x0_ 6': [4.697517762, 6.01930508, 3.48788064, 4.31676536, 5.16289777, 8.154419, 5.2342344, 4.730160786, 4.707832836, 6.32186544], 'x0_ 8': [97.554, 119.18, 101.28, 108.88, 113.09, 113.0, 111.9, 82.914, 84.982, 108.24], 'x0_ 9': [97.554, 119.18, 101.28, 108.88, 113.09, 113.0, 111.9, 82.914, 84.982, 108.24], 'x0_10': [3.19814917262, 4.96870842374, 5.27738007224, 3.95608314762, 4.63464033793, 1.7150416755, 4.7237153349, 2.58030593927, 4.62123641991, 3.58883085798], 'x0_11': [57.951224472, 77.8484760287, 80.7969837316, 97.383030189, 83.8603979178, 79.9949521754, 94.4307701512, 64.7483947298, 71.2301660032, 93.319683293], 'x0_12': [93.2258828462, 47.1721248798, 81.3429092974, 124.746856854, 71.5419213875, 52.

In [23]:
print(len(param_dict))

50


In [24]:
# import multivariate_normal
from scipy.stats import multivariate_normal

data = np.array([param_dict[key] for key in param_dict])
print(data)
# mean and covariance
mean = np.mean(data, axis=1)
cov = np.cov(data)
cov = cov + np.eye(cov.shape[0]) * 1e-4

print(mean.shape)
print(cov.shape)
# create multivariate normal
# convert to tensor
mean = torch.tensor(mean, dtype=torch.float32, device=device)
cov = torch.tensor(cov, dtype=torch.float32, device=device)

# import torch distributions
from torch.distributions.multivariate_normal import MultivariateNormal

# create multivariate normal
mvn = MultivariateNormal(loc=mean, covariance_matrix=cov)

[[2.50621836e+02 2.80236267e+02 3.26553380e+02 2.48117090e+02
  2.79047113e+02 2.57421705e+02 2.38918844e+02 2.63838120e+02
  2.41447500e+02 2.67183741e+02]
 [1.76506560e+02 3.09541309e+01 3.83368339e+02 2.14758608e+02
  1.44715759e+02 1.62558571e+02 8.79615903e+01 3.39274151e+01
  1.84846173e+02 1.69711073e+02]
 [4.69751776e+00 6.01930508e+00 3.48788064e+00 4.31676536e+00
  5.16289777e+00 8.15441900e+00 5.23423440e+00 4.73016079e+00
  4.70783284e+00 6.32186544e+00]
 [9.75540000e+01 1.19180000e+02 1.01280000e+02 1.08880000e+02
  1.13090000e+02 1.13000000e+02 1.11900000e+02 8.29140000e+01
  8.49820000e+01 1.08240000e+02]
 [9.75540000e+01 1.19180000e+02 1.01280000e+02 1.08880000e+02
  1.13090000e+02 1.13000000e+02 1.11900000e+02 8.29140000e+01
  8.49820000e+01 1.08240000e+02]
 [3.19814917e+00 4.96870842e+00 5.27738007e+00 3.95608315e+00
  4.63464034e+00 1.71504168e+00 4.72371533e+00 2.58030594e+00
  4.62123642e+00 3.58883086e+00]
 [5.79512245e+01 7.78484760e+01 8.07969837e+01 9.73830302e

In [27]:
samples = mvn.sample((10,))

# 50 parameters
print(samples[0].shape)

torch.Size([50])
